In [ ]:
import sys
# TO CHANGE
BASEDIR = "../../../.."
sys.path.insert(0, BASEDIR)

# TODO (Ноутбук недоделан)

In [ ]:
import pprint

In [ ]:
from src.kg_model import KnowledgeGraphModel, KnowledgeGraphModelConfig
from src.db_drivers.vector_driver import EmbedderModelConfig
from src.pipelines.memorize import MemPipelineConfig, MemPipeline, LLMUpdatorConfig

from src.utils.data_structs import QueryInfo
from src.pipelines.qa.kg_reasoning.weak_reasoner.knowledge_retriever import KnowledgeRetriever, KnowledgeRetrieverConfig
from src.pipelines.qa.kg_reasoning.weak_reasoner.knowledge_retriever import WaterCirclesSearchConfig, MixturedGraphSearchConfig, AStarGraphSearchConfig, \
    NaiveBFSGraphSearchConfig, NaiveGraphSearchConfig, GraphBeamSearchConfig, TripletsFilterConfig

/home/dzigen/Desktop/Projects/PersonalAI/.pai_venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Пример настройки алгоритма обхода графа/извлечения информации в рамках QA-пайплайна

1. Инициализация модели графа знаний

In [ ]:
kg_config = KnowledgeGraphModelConfig(
    nodestree_config=None # Модель дерева вершин строиться не будет
)

EMBEDDER_MODEL_PATH = '../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL
kg_config.embedders_config['m-e5-small'] = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)
kg_model = KnowledgeGraphModel(kg_config)

No sentence-transformers model found with name ../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


2. Инициализация Memorize-пайплайна

In [ ]:
mem_config = MemPipelineConfig(
    updator_config=LLMUpdatorConfig(
        delete_obsolete_info=False # Выключаем механизм по поиску/удалению устревших знаний в графе при добавлении новой информации
    )
)
mem_pipeline = MemPipeline(kg_model, mem_config)

3. Добавление информации в граф знаний

In [ ]:
TEXT_EXAMPLES = [
    "Sasha was walking along the highway.", 
    "Masha was walking along the highway.", 
    "The ship was sailing along the water canal.", 
    "The motorboat was sailing along the river."]

In [ ]:
extracted_triplets = []
for example in TEXT_EXAMPLES:
    tmp_extracted_triplets, _ = mem_pipeline.remember(example)
    extracted_triplets += tmp_extracted_triplets

In [ ]:
kg_model.count_items()

{'graph_info': {'triplets': 31, 'nodes': 19},
 'embeddings_info': {'nodes': 19, 'triplets': 14},
 'nodestree_info': None}

Всего доступно 6 алгоритмов обхода графа:
* beamsearch
* astart
* watercircles
* naive_bfs
* naive_retriever
* mixture (комбинация двух базовых алгоритмов)

А также доступен один алгоритм фильтрации извлечённых треплетов (после обхода графа):
* naive

4.1. Инициализация KnowledgeRetriever-шага с 'beamsearch'-обходом и 'naive'-фильтрацией

In [ ]:
kr_config = KnowledgeRetrieverConfig(
    retriever_method='beamsearch',
    retriever_config=GraphBeamSearchConfig(),
    filter_method='naive',
    filter_config=TripletsFilterConfig()
)
pprint(kr_config)

In [ ]:
k_retriever = KnowledgeRetriever(kg_model, kr_config)

In [ ]:
EXAMPLE_QUERY = QueryInfo(query="Have motorboat was ever sailed through a water canal?")

In [ ]:
relevant_triplets, rinfo = k_retriever.retrieve()